In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from matplotlib import rc
plt.style.use('classic')
rc('font', family='serif')
from astropy.io import fits, ascii
from astropy.table import Table
import os
import math

import TheCannon
from TheCannon import apogee, dataset, model
import apogee_tools as ap
AP_PATH = os.environ['APOGEE_DATA']

line_list = ap.lines
line_names = line_list.keys()

/home/jess/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [47]:
dfile = AP_PATH + '/training_sets/mann_sample/'
rfile = 'train_ref/mann_ref_labels.csv'  

ids, wl, tr_flux, tr_ivar = apogee.load_spectra(dfile)
tr_ID = [x.split('2-')[1].split('.fits')[0] for x in ids]
tr_label = scaleLabels(ap.loadLabels(rfile, lbl_names=['TEFF', 'FE_H']))

ds = dataset.Dataset(wl, tr_ID, tr_flux, tr_ivar, tr_label, tr_ID, tr_flux, tr_ivar)

This is the right version
Loading spectra from directory /home/jess/Desktop/Research/apogee_data/training_sets/mann_sample/
Spectra loaded
Loading dataset
This may take a while...


In [4]:
def jackknife(ds, **kwargs):

    """
    Cross-validation test of Cannon test spectra
    Input:  'ds' : dataset object output from initializeTrainingSet()
    Output: plot training label vs. inferred label left out of set
            return training label and inferred label
    """

    # optional
    label_names = kwargs.get('lbl_names', ['Teff', 'Fe/H'])

    # ds: Data set of all objects (including n); continuum normalized in initializeTrainingSet()
    wl = ds.wl
    nsources = len(ds.tr_ID)

    # Training labels and cross-validated labels
    trn_labels, crv_labels = [], []

    # Remove nth spectrum from the training set and run Cannon model on the rest of spectra
    for n in range(nsources): 

        tr_ids   = list(ds.tr_ID)
        tr_flux  = list(ds.tr_flux)
        tr_ivar  = list(ds.tr_ivar)
        tr_label = list(ds.tr_label)

        tr_ids.pop(n)
        tr_flux.pop(n)
        tr_ivar.pop(n)
        tr_label.pop(n)

        id_minus_n = np.array(tr_ids)
        fl_minus_n = np.array(tr_flux)
        vr_minus_n = np.array(tr_ivar)
        tr_label_minus_n = np.array(tr_label)


        ds_minus_n = dataset.Dataset(wl, id_minus_n, fl_minus_n, vr_minus_n, tr_label_minus_n, \
            id_minus_n, fl_minus_n, vr_minus_n)

        ds_minus_n.set_label_names(label_names)       
        model_minus_n, synth_fl_minus_n, te_label_minus_n = ap.runCannon(ds_minus_n)

        # Find cross-validation label
        crv_label_n = test_labels[n]
        crv_labels.append(crv_label_n)
        
        # Jackknife
        
        
        np.save(model_minus_n.coeffs, save_dir+'model_coeff_'+str(i))
        np.save(synth_fl_minus_n, save_dir+'synth_flux_'+str(i))
        np.save(te_label_minus_n, save_dir+'test_labels_'+str(i))
        np.save(crv_label_n, save_dir+'crv_label_'+str(i))

        print('Labeled [%s/%s] sources.\n'%(n+1, nsources))

    trn_labels = ds.tr_label

    return trn_labels, crv_labels

In [37]:
def _getPivotsAndScales(label_vals):

    qs = np.percentile(label_vals, (2.5, 50, 97.5), axis=0)
    pivots = qs[1]
    scales = (qs[2] - qs[0])/4.

    return pivots, scales

def scaleLabels(labels):
    
    scaled_labels = []
    for i in range(labels.shape[1]):
        p, s = _getPivotsAndScales(labels.T[i])
        slbl = [(t - p)/s for t in labels.T[i]]
        scaled_labels.append(slbl)
        
    return np.array(scaled_labels).T

In [15]:
tr_label.shape[1]

2